To start, define the user-specific constants:

In [1]:
'''
These parameters describe the FRB that you want results for. Example:

FRB_Name = 'Spock'
FRB_Ra = 323.477
FRB_Dec = 13.244
host_redshift = 0.367
'''

FRB_Name = 'Whitney'
FRB_Ra = 134.7205	 #in degrees
FRB_Dec = 73.4908 #in degrees
host_redshift = 0.479 #float

'''
Put your Casjobs user ID here. If you do not have one, you can make an account at: https://mastweb.stsci.edu/ps1casjobs/CreateAccount.aspx
'''

CASJOBS_WSID_string = '859089964' #string

Now let's import dependencies and input the FRB information:

In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.table import Row
from astropy.io import fits
from astropy.utils.data import download_file
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus, ApertureStats
from astropy.cosmology import WMAP9 as cosmo
import matplotlib
from matplotlib import pyplot as plt
matplotlib.use('TkAgg')
from numpy import *
from pylab import *
from scipy import stats
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy import coordinates, units
from astropy.io import ascii
import mastcasjobs
from psquery import mastquery
import getpass
import os
os.environ["CASJOBS_WSID"] = CASJOBS_WSID_string
import re
from psquery import get_coord
import time, sys, os
import h5py
import numpy as np
import scipy
import fsps
import sedpy
import prospect
import emcee
import dynesty
from matplotlib.pyplot import *

%matplotlib inline

# re-defining plotting defaults
from matplotlib.font_manager import FontProperties
from matplotlib import gridspec

noaodatalab not imported. cannot query legacysurvey...


In [3]:
sizestring = '2400'
size = 2400
radius = 15
cone = 300/3600 #in degrees
format = 'fits'

Let's set up our output files.

In [4]:
file = open(FRB_Name+' Results.txt', 'w')
file.write(FRB_Name+' coordinates: ('+str(FRB_Ra)+', '+str(FRB_Dec)+')\n\n')
file.close()

Now, let's define the file reading functions:

In [5]:
def geturl(ra, dec, size, filter):

    '''
    Function that creates a PanStarrs FITS url to download.

    Parameters:
    -----------
    ra  : float
        J2000 RA of center of image in degrees

    dec : float
        J2000 Dec of center of image in degrees

    size : integer
        image size in pixels where 240 pixels is 60 arcsec

    filter : string
        desired filter for the image (g, r, i, z, or y)

    Returns:
    --------
    url : string
        desired url to unpack for a FITS file
    '''
    
    format, output_size, color = "fits", None, False

    service = "https://ps1images.stsci.edu/cgi-bin/ps1filenames.py"
    
    url = ("{service}?ra={ra}&dec={dec}&size={size}&format=fits"
           "&filters={filter}").format(**locals())
    table = Table.read(url, format='ascii')
    
    url = ("https://ps1images.stsci.edu/cgi-bin/fitscut.cgi?"
           "ra={ra}&dec={dec}&size={size}&format={format}").format(**locals())

    urlbase = url + "&red="
    url = []
    for filename in table['filename']:
        url.append(urlbase+filename)
        
    return url

def readfile(ra, dec, size, filter):

    '''
    This function runs the get_url() function, creates a path, and downloads the FITS file data and header. 

    Parameters:
    -----------
    ra  : float
        J2000 RA of center of image in degrees

    dec : float
        J2000 Dec of center of image in degrees

        size : integer
        image size in pixels where 240 pixels is 60 arcsec

    filter : string
                desired filter for the image (g, r, i, z, or y)

    Returns:
    -------
    data : numpy.ndarray
        data describing the FITS file

    header: astropy.io.fits.header.Header
            header for the FITS file containing information about the collection processes and image reduction
    '''
   
    url=geturl(ra, dec, size, filter)[0]
    path = download_file(url)
    hdu = fits.open(path)
    data = hdu[0].data
    header = hdu[0].header
    hdu.close()

    return [data, header]


We're going to run a query in the STRM database to pull surrounding galaxies:

In [6]:
'''
This cell uses the mastquery function from Casey Law's psquery package.

Parameters:
-----------

FRB_Ra : float
         RA defined above, in degrees

FRB_Dec : float
          Dec defined above, in degrees

cone : float
       radius of cone search defined above, in degrees

selectcol : list
            desired information from STRM, out of ["objID", "raMean", "decMean", "z_phot0", "z_photErr", "prob_Galaxy", "prob_Star", "prob_QSO"]

Returns:
--------

STRM_results : astropy table
               table of results from the mastquery search

intervening_coords : list
                     this is the list created in the STRM mastquery cell
                     it has entries with the following form: 
                     [STRM_results[i]['raMean'], STRM_results[i]['decMean'], STRM_results[i]['z_phot0'], STRM_results[i]['z_photErr']]

'''

deg_in_rad = 57.2958
virial_radius_mpc = 0.3

STRM_results = mastquery.cone_ps1strm((FRB_Ra, FRB_Dec), cone, selectcol=["raMean", "decMean", "z_phot0", "z_photErr", "prob_Galaxy", "prob_Star"])
intervening_coords = []
i = 0
while i < len(STRM_results):
    FRB_coord = SkyCoord(FRB_Ra*units.deg, FRB_Dec*units.deg, frame='icrs')
    temp_coord = SkyCoord(STRM_results[i]['raMean']*units.deg, STRM_results[i]['decMean']*units.deg, frame='icrs')
    separation = (FRB_coord.separation(temp_coord)).deg
    d_mpc = (cosmo.angular_diameter_distance(z=(STRM_results[i]['z_phot0'])-2*STRM_results[i]['z_photErr'])).to_value()
    theta_virial = 57.2958*(0.3/d_mpc)
    #let's get rid of objects that probably aren't galaxies
    if STRM_results[i]['prob_Galaxy'] < 0.5:
        STRM_results.remove_row(i)
    elif STRM_results[i]['prob_Star'] > 0.5:
        STRM_results.remove_row(i)
    #any object with a separation larger than the largest Virial radius is not at risk of intervening
    elif separation > theta_virial:
        STRM_results.remove_row(i)
    #any object more distant than the FRB host is not at risk of intervening
    #we use the smallest possible extreme of the zphot
    elif STRM_results[i]['z_phot0'] - 2*STRM_results[i]['z_photErr'] > host_redshift:
        STRM_results.remove_row(i)
    #we assume that any object without a real zphot detection is not at risk of intervening
    elif STRM_results[i]['z_phot0'] < 0:
        STRM_results.remove_row(i)
    else:
        intervening_coords.append([STRM_results[i]['raMean'], STRM_results[i]['decMean'], STRM_results[i]['z_phot0'], STRM_results[i]['z_photErr'], separation])
        i+=1

/home/rns/.local/lib/python3.8/site-packages/astropy/cosmology/flrw/base.py:1076: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return quad(self._inv_efunc_scalar, z1, z2, args=self._inv_efunc_scalar_args)[0]


This function runs the aperture photometry functions from photutils and calculates the error. 

The magnitude is found by the following equation:
$$
M = M_{ZPT} - 2.5\log_{10}{\frac{c \cdot g}{t_{exp}}} 
$$
...where $M_{ZPT}$ is the zero-point magnitude (25 for PanStarrs FITS files), $t_{exp}$ is the exposure time in seconds, $c$ is the counts per pixel, and $g$ is the photon-electron gain.

The error in this magnitude can be approximated by:
$$
\sigma_{M} \approx \frac{-2.5}{\ln{10} \cdot c} \sqrt{\left(A_{aperture} + \frac{A_{aperture}^2}{A_{BG}}\right)\sigma_{BG}^2}
$$

And the 3-sigma upper limit on non-detections is then found with:

$$
M_{3\sigma} = M_{ZPT} - 2.5\log{\left[\mu_{BG}+3\sigma_{BG}*\frac{g}{t_{exp}}\right]}\pm{\left|\frac{2.5}{\ln{10} \cdot c}\right|\sqrt{\sigma_{BG}^2\left(A_{aperture}+\frac{A_{aperture}^2}{A_{BG}}\right)}}
$$

In [152]:
def getmagnitude(data, header, radius, position):

    '''
    Function that handles the aperture photometry and calculates associated error. 

    Parameters:
    ----------
    data : numpy.ndarray
           data describing the FITS file

    header : astropy.io.fits.header.Header
             header for the FITS file containing information about the collection processes and image reduction

    radius : integer
             radius of aperture in pixels

    position : list 
               contains x and y coordinates (in pixels) of the center of the aperture

    Returns:
    --------
    mag_and_error : list
                    contains either the magnitude and error or the 3-sigma upper limit and the error
                    if there is no detection, the error will be negative
    
    '''

    Texp = header['EXPTIME']
    ZPTmag = header['HIERARCH FPA.ZP']
    gain = header['HIERARCH CELL.GAIN']

    aperture = CircularAperture(position, radius)
    annulus = CircularAnnulus(position, radius+2, np.sqrt(2*(radius**2+2*radius+2))) 

    phot_table = aperture_photometry(data, aperture)
    ann_table = aperture_photometry(data, annulus)

    counts = phot_table['aperture_sum'] - (ann_table['aperture_sum'])*(aperture.area/annulus.area)
    
    annstats  = ApertureStats(data, annulus)
    annvar = (annstats.std)**2

    count_error = np.sqrt(annvar*(aperture.area+(aperture.area**2)/annulus.area))
    error = (2.5/(np.log(10)*counts))*count_error
    upperlimcounts = annstats.mean+3*annstats.std
    Ne = counts*gain
    
    if Ne/Texp > 0:
        magnitude = ZPTmag - 2.5*np.log10(Ne/Texp)
        return [magnitude[0], error[0]]
    else:
        Ne = ann_table['aperture_sum']*gain
        magnitude = ('3σ: '+str("{:.4f}".format(ZPTmag - 2.5*np.log10(upperlimcounts*gain/Texp))))
        return [magnitude, str("{:.4f}".format(error[0]))]


In [153]:
def getinfo(FRB_Name, FRBra, FRBdec, intervening_coords):

    '''
    Function that runs the get_magnitude function for a number of coordinates in the same field of view. 

    Parameters:
    -----------

    FRB_Name : string
               name of FRB defined above

    FRB_Ra : float
         RA defined above, in degrees

    FRB_Dec : float
          Dec defined above, in degrees

    intervening_coords : list
                         this is the list created in the STRM mastquery cell
                         it has entries with the following form: 
                         [STRM_results[i]['raMean'], STRM_results[i]['decMean'], STRM_results[i]['z_phot0'], STRM_results[i]['z_photErr']]

    Returns: 
    --------

    df : Pandas DataFrame
         this contains all of the aperture photometry detections in each bandpass for each candidate
    
    '''
    Name, Ra, Dec, Separation, zphot, zphot_err = [], [], [], [], [], []
    filterdict = {'g':[], 'r':[], 'i':[], 'z':[], 'y':[]}
    errordict = {'err_g':[], 'err_r':[], 'err_i':[], 'err_z':[], 'err_y':[]}
    
    j = 0
    for coord in intervening_coords:
        Name.append(FRB_Name+str(j))
        ratemp = coord[0]
        dectemp = coord[1]
        zphot_temp = coord[2]
        zphot_err_temp = coord[3]
        separation_temp = coord[4]
        Ra.append(ratemp)
        Dec.append(dectemp)
        zphot.append(zphot_temp)
        zphot_err.append(zphot_err_temp)
        Separation.append(separation_temp)
        j+=1

    for filter in filterdict:
        file = readfile(FRBra, FRBdec, size, filter)
        data = file[0]
        header = file[1]
        w = WCS(header)

        for coord in intervening_coords:
            ratemp = coord[0]
            dectemp = coord[1]  
            sky = SkyCoord(ra = ratemp, dec = dectemp, unit='deg')
            position = w.world_to_pixel(sky)
            mag_and_error = getmagnitude(data, header, radius, position)
            filterdict[filter].append(mag_and_error[0])
            errordict['err_'+filter].append(mag_and_error[1])


    content = {'Name': Name, 'RA': Ra, 'Dec': Dec, 'Separation': Separation, 'zphot': zphot, 'zphot_err': zphot_err,
               'g': filterdict['g'], 'err_g' : errordict['err_g'],
               'r': filterdict['r'], 'err_r' : errordict['err_r'],
               'i': filterdict['i'], 'err_i' : errordict['err_i'],
               'z': filterdict['z'], 'err_z' : errordict['err_z'],
               'y': filterdict['y'], 'err_y' : errordict['err_y']}


    df = pd.DataFrame(content)

    return df

Now we call a bunch of functions to perform aperture photometry on a wide range of candidates:

In [154]:
intervening_info = {'Name':[],
                        'RA':[], 
                        'Dec':[], 
                        'Separation':[],
                        'g':[],
                        'err_g':[],
                        'r':[],
                        'err_r':[],
                        'i':[],
                        'err_i':[],
                        'z':[],
                        'err_z':[],
                        'y':[],
                        'err_y':[]}

df = getinfo(FRB_Name, FRB_Ra, FRB_Dec, intervening_coords)


this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]

Let's try to identify the detection that corresponds to the FRB host galaxy, and compare this to our known redshift.

In [155]:
separations = df['Separation'].to_list()
lowest_separation = min(separations)
lowest_separation_index = np.argmin(separations)
lowest_zphot = df.at[lowest_separation_index, 'zphot']
lowest_zphot_err = df.at[lowest_separation_index, 'zphot_err']
lowest_RA = df.at[lowest_separation_index, 'RA']
lowest_Dec = df.at[lowest_separation_index, 'Dec']

print('Photometric redshift: '+str(lowest_zphot))
print('Photometric redshift error: '+str(lowest_zphot_err))
print('Spectroscopic redshift: '+str(host_redshift))

file = open(FRB_Name+' Results.txt', 'a')
file.write('Below is the information for the source listed in STRM with the smallest separation from the localized FRB.\n')
file.write('It is possible that STRM will not have usable data for this detection, that the object is not real, or that it is not the host galaxy.\n\n')
file.write('The closest detection was an object at ('+str(lowest_RA)+', '+str(lowest_Dec)+') with the following information:\n')
file.write('Photometric redshift: '+str(lowest_zphot))
file.write('\nPhotometric redshift: '+str(lowest_zphot_err))
file.write('\nCompare to the spectroscopic redshift: '+str(host_redshift))
file.close()

Photometric redshift: 0.49400535
Photometric redshift error: 0.0762875424375022
Spectroscopic redshift: 0.479


Now let's get rid of the incomplete detections: 

In [156]:
idlist = ['Name', 'RA', 'Dec', 'Separation', 'zphot', 'zphot_err', 'g', 'err_g', 'r', 'err_r', 'i', 'err_i', 'z', 'err_z', 'y', 'err_y']

detection_names = []
non_detection_names = []

detections = pd.DataFrame(columns = idlist)
non_detections = pd.DataFrame(columns = idlist)

for i in range(0, len(df)):
    for f in ['g', 'r', 'i', 'z', 'y']:
        #we place aside objects that might intervene but are not detected in all bandpasses
        if type(df.at[i,f]) == str:
            non_detection_names.append(FRB_Name+str(i))
            break
        elif df.at[i,f] > 25.0:
            non_detection_names.append(FRB_Name+str(i))
            break
        #we consider the rest to be viable, detected candidates
        elif f == 'y':
            detection_names.append(FRB_Name+str(i))
        else:
            continue

for name in detection_names:
    line = pd.DataFrame([df.iloc[int(name.lstrip(FRB_Name))].to_list()], columns = idlist)
    detections = pd.concat([detections, line])

for name in non_detection_names:
    line = pd.DataFrame([df.iloc[int(name.lstrip(FRB_Name))].to_list()], columns = idlist)
    non_detections = pd.concat([non_detections, line])

detections = detections.set_index(pd.Index(range(0, len(detections))))
non_detections = non_detections.set_index(pd.Index(range(0, len(non_detections))))

print(len(detections))
print(len(non_detections))
print(detections)

file = open(FRB_Name+' Results.txt', 'a')
file.write('\n\nThe following sources were detected in all bandpasses and might intersect the FRB line of sight:\n\n')
for id in idlist:
    file.write(id+'   ')
file.write('\n')
file.close()
detections.to_csv(FRB_Name+' Results.txt', header=None, index=None, sep=' ', mode='a', float_format='%.4f')
file = open(FRB_Name+' Results.txt', 'a')
file.write('\n\nThe following sources were not detected in all bandpasses but might still intersect the FRB line of sight:\n\n')
for id in idlist:
    file.write(id+'   ')
file.write('\n')
file.close()
non_detections.to_csv(FRB_Name+' Results.txt', header=None, index=None, sep=' ', mode='a', float_format = '%.4f')

19
69
         Name          RA        Dec  Separation     zphot  zphot_err  \
0    Whitney1  134.586313  73.543718    0.065191  0.605416   0.293540   
1    Whitney4  134.460778  73.518832    0.078892  1.023200   0.490557   
2   Whitney16  134.614383  73.435290    0.063195  0.225227   0.105161   
3   Whitney17  134.754099  73.434348    0.057256  0.121304   0.043535   
4   Whitney21  134.535645  73.449391    0.066939  0.514999   0.248327   
5   Whitney26  134.697478  73.472627    0.019316  0.909463   0.404891   
6   Whitney28  134.680142  73.475089    0.019455  0.440621   0.159480   
7   Whitney33  134.549191  73.501826    0.049898  0.654853   0.285203   
8   Whitney34  134.574410  73.523126    0.052585  0.328621   0.136607   
9   Whitney37  134.765135  73.541053    0.051824  0.464585   0.219268   
10  Whitney39  134.817213  73.544766    0.060541  0.615271  14.143024   
11  Whitney44  134.813922  73.465792    0.036486  0.592470   0.253288   
12  Whitney45  134.753342  73.471311    0.021

Now it's time to set up some imports and functions for the SED fitting (taken straight from the Prospector interactive demo)

In [157]:
rcParams.update({'xtick.major.pad': '7.0'})
rcParams.update({'xtick.major.size': '7.5'})
rcParams.update({'xtick.major.width': '1.5'})
rcParams.update({'xtick.minor.pad': '7.0'})
rcParams.update({'xtick.minor.size': '3.5'})
rcParams.update({'xtick.minor.width': '1.0'})
rcParams.update({'ytick.major.pad': '7.0'})
rcParams.update({'ytick.major.size': '7.5'})
rcParams.update({'ytick.major.width': '1.5'})
rcParams.update({'ytick.minor.pad': '7.0'})
rcParams.update({'ytick.minor.size': '3.5'})
rcParams.update({'ytick.minor.width': '1.0'})
rcParams.update({'xtick.color': 'k'})
rcParams.update({'ytick.color': 'k'})
rcParams.update({'font.size': 30})

In [158]:
def build_obs(snr=10, ldist=10.0, **extras):
    """Build a dictionary of observational data.  In this example 
    the data consist of photometry for a single nearby dwarf galaxy 
    from Johnson et al. 2013.
    
    :param snr:
        The S/N to assign to the photometry, since none are reported 
        in Johnson et al. 2013
        
    :param ldist:
        The luminosity distance to assume for translating absolute magnitudes 
        into apparent magnitudes.
        
    :returns obs:
        A dictionary of observational data to use in the fit.
    """
    from prospect.utils.obsutils import fix_obs
    import sedpy

    # The obs dictionary, empty for now
    obs = {}

    # These are the names of the relevant filters, 
    # in the same order as the photometric data (see below)
    # Now we store the measured fluxes for a single object, **in the same order as "filters"**
    # In this example we use a row of absolute AB magnitudes from Johnson et al. 2013 (NGC4163)
    # We then turn them into apparent magnitudes based on the supplied `ldist` meta-parameter.
    # You could also, e.g. read from a catalog.
    # The units of the fluxes need to be maggies (Jy/3631) so we will do the conversion here too.
    panstarrs = ['ps_g', 'ps_r','ps_i', 'ps_z', 'ps_y']
    filternames = panstarrs
    # And here we instantiate the `Filter()` objects using methods in `sedpy`,
    # and put the resultinf list of Filter objects in the "filters" key of the `obs` dictionary
    obs["filters"] = sedpy.observate.load_filters(filternames)
    mags = np.array(galaxy_magnitudes)
    # print(mags)
    obs["maggies"] = 10**(-0.4*mags)

    # And now we store the uncertainties (again in units of maggies)
    # In this example we are going to fudge the uncertainties based on the supplied `snr` meta-parameter.
    errors = np.array(galaxy_errors)
    obs["maggies_unc"] = 0.4*errors*10**(-0.4*mags)*np.log(10)

    # Now we need a mask, which says which flux values to consider in the likelihood.
    # IMPORTANT: the mask is *True* for values that you *want* to fit, 
    # and *False* for values you want to ignore.  Here we ignore the spitzer bands.
    obs["phot_mask"] = np.array(['spitzer' not in f.name for f in obs["filters"]])

    # This is an array of effective wavelengths for each of the filters.  
    # It is not necessary, but it can be useful for plotting so we store it here as a convenience
    obs["phot_wave"] = np.array([f.wave_effective for f in obs["filters"]])

    # We do not have a spectrum, so we set some required elements of the obs dictionary to None.
    # (this would be a vector of vacuum wavelengths in angstroms)
    obs["wavelength"] = None
    # (this would be the spectrum in units of maggies)
    obs["spectrum"] = None
    # (spectral uncertainties are given here)
    obs['unc'] = None
    # (again, to ignore a particular wavelength set the value of the 
    #  corresponding elemnt of the mask to *False*)
    obs['mask'] = None

    # This function ensures all required keys are present in the obs dictionary,
    # adding default values if necessary
    obs = fix_obs(obs)

    return obs

In [159]:
def build_model(object_redshift=None, ldist=10.0, fixed_metallicity=None, add_duste=False, 
                **extras):
    """Build a prospect.models.SedModel object
    
    :param object_redshift: (optional, default: None)
        If given, produce spectra and observed frame photometry appropriate 
        for this redshift. Otherwise, the redshift will be zero.
        
    :param ldist: (optional, default: 10)
        The luminosity distance (in Mpc) for the model.  Spectra and observed 
        frame (apparent) photometry will be appropriate for this luminosity distance.
        
    :param fixed_metallicity: (optional, default: None)
        If given, fix the model metallicity (:math:`log(Z/Z_sun)`) to the given value.
        
    :param add_duste: (optional, default: False)
        If `True`, add dust emission and associated (fixed) parameters to the model.
        
    :returns model:
        An instance of prospect.models.SedModel
    """
    from prospect.models.sedmodel import SedModel
    from prospect.models.templates import TemplateLibrary
    from prospect.models import priors

    # Get (a copy of) one of the prepackaged model set dictionaries.
    # This is, somewhat confusingly, a dictionary of dictionaries, keyed by parameter name
    model_params = TemplateLibrary["parametric_sfh"]
    
   # Now add the lumdist parameter by hand as another entry in the dictionary.
   # This will control the distance since we are setting the redshift to zero.  
   # In `build_obs` above we used a distance of 10Mpc to convert from absolute to apparent magnitudes, 
   # so we use that here too, since the `maggies` are appropriate for that distance.
    model_params["lumdist"] = {"N": 1, "isfree": False, "init": ldist, "units":"Mpc"}
    
    # Let's make some changes to initial values appropriate for our objects and data
    model_params["zred"]["init"] = galaxy_redshift
    model_params["dust2"]["init"] = 0.05
    model_params["logzsol"]["init"] = -0.5
    model_params["tage"]["init"] = 13.
    model_params["mass"]["init"] = 1e8
    
    # These are dwarf galaxies, so lets also adjust the metallicity prior,
    # the tau parameter upward, and the mass prior downward
    model_params["dust2"]["prior"] = priors.TopHat(mini=0.0, maxi=2.0)
    model_params["tau"]["prior"] = priors.LogUniform(mini=1e-1, maxi=1e2)
    model_params["mass"]["prior"] = priors.LogUniform(mini=1e6, maxi=1e10)

    # If we are going to be using emcee, it is useful to provide a 
    # minimum scale for the cloud of walkers (the default is 0.1)
    model_params["mass"]["disp_floor"] = 1e6
    model_params["tau"]["disp_floor"] = 1.0
    model_params["tage"]["disp_floor"] = 1.0
    
    # Change the model parameter specifications based on some keyword arguments
    if fixed_metallicity is not None:
        # make it a fixed parameter
        model_params["logzsol"]["isfree"] = False
        #And use value supplied by fixed_metallicity keyword
        model_params["logzsol"]['init'] = fixed_metallicity 

    if object_redshift is not None:
        # make sure zred is fixed
        model_params["zred"]['isfree'] = False
        # And set the value to the object_redshift keyword
        model_params["zred"]['init'] = galaxy_redshift

    if add_duste:
        # Add dust emission (with fixed dust SED parameters)
        # Since `model_params` is a dictionary of parameter specifications, 
        # and `TemplateLibrary` returns dictionaries of parameter specifications, 
        # we can just update `model_params` with the parameters described in the 
        # pre-packaged `dust_emission` parameter set.
        model_params.update(TemplateLibrary["dust_emission"])
        
    # Now instantiate the model object using this dictionary of parameter specifications
    model = SedModel(model_params)

    return model

In [160]:
def build_sps(zcontinuous=1, **extras):
    """
    :param zcontinuous: 
        A vlue of 1 insures that we use interpolation between SSPs to 
        have a continuous metallicity parameter (`logzsol`)
        See python-FSPS documentation for details
    """
    from prospect.sources import CSPSpecBasis
    sps = CSPSpecBasis(zcontinuous=zcontinuous)
    return sps

In [161]:
from prospect.likelihood import chi_spec, chi_phot
def chivecfn(theta):
    """A version of lnprobfn that returns the simple uncertainty 
    normalized residual instead of the log-posterior, for use with 
    least-squares optimization methods like Levenburg-Marquardt.
    
    It's important to note that the returned chi vector does not 
    include the prior probability.
    """
    lnp_prior = model.prior_product(theta)
    if not np.isfinite(lnp_prior):
        return np.zeros(model.ndim) - np.infty

    # Generate mean model
    try:
        spec, phot, x = model.mean_model(theta, obs, sps=sps)
    except(ValueError):
        return np.zeros(model.ndim) - np.infty

    chispec = chi_spec(spec, obs)
    chiphot = chi_phot(phot, obs)
    return np.concatenate([chispec, chiphot])

In [166]:
def plot_SED(wphot, obs):

    '''
    whitney runtime: about 5 min
    '''
    xmin, xmax = np.min(wphot)*0.8, np.max(wphot)/0.8
    ymin, ymax = obs["maggies"].min()*0.8, obs["maggies"].max()/0.4
    figure(figsize=(16,8))

    # plot all the data
    plot(wphot, obs['maggies'],
        label='All observed photometry',
        marker='o', markersize=12, alpha=0.8, ls='', lw=3,
        color='slateblue')

    # overplot only the data we intend to fit
    mask = obs["phot_mask"]
    errorbar(wphot[mask], obs['maggies'][mask], 
            yerr=obs['maggies_unc'][mask], 
            label='Photometry to fit',
            marker='o', markersize=8, alpha=0.8, ls='', lw=3,
            ecolor='tomato', markerfacecolor='none', markeredgecolor='tomato', 
            markeredgewidth=3)

    # plot Filters
    for f in obs['filters']:
        w, t = f.wavelength.copy(), f.transmission.copy()
        t = t / t.max()
        t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
        loglog(w, t, lw=3, color='gray', alpha=0.7)

    # prettify
    xlabel('Wavelength [A]')
    ylabel('Flux Density [maggies]')
    xlim([xmin, xmax])
    ylim([10e-11, ymax])
    xscale("log")
    yscale("log")
    legend(loc='best', fontsize=20)
    tight_layout()

In [167]:
def plot_prelim_fit(wphot, wspec, initial_spec, obs):

    ''' 
    whitney runtime: about 20 min
    '''

    #establish bounds
    xmin, xmax = np.min(wphot)*0.8, np.max(wphot)/0.8
    temp = np.interp(np.linspace(xmin,xmax,10000), wspec, initial_spec)
    ymin, ymax = temp.min()*0.8, temp.max()/0.4
    figure(figsize=(16,8))

    # plot model + data
    loglog(wspec, initial_spec, label='Model spectrum', 
        lw=0.7, color='navy', alpha=0.7)
    errorbar(wphot, initial_phot, label='Model photometry', 
            marker='s',markersize=10, alpha=0.8, ls='', lw=3,
            markerfacecolor='none', markeredgecolor='blue', 
            markeredgewidth=3)
    errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'], 
            label='Observed photometry',
            marker='o', markersize=10, alpha=0.8, ls='', lw=3,
            ecolor='red', markerfacecolor='none', markeredgecolor='red', 
            markeredgewidth=3)
    title(title_text)

    # plot Filters
    for f in obs['filters']:
        w, t = f.wavelength.copy(), f.transmission.copy()
        t = t / t.max()
        t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
        loglog(w, t, lw=3, color='gray', alpha=0.7)

    #prettify
    xlabel('Wavelength [A]')
    ylabel('Flux Density [maggies]')
    xlim([xmin, xmax])
    ylim([10e-11, ymax])
    legend(loc='best', fontsize=20)
    tight_layout()

In [168]:
def plot_least_squares(model, theta_best, obs, sps, wspec, wphot, initial_spec, initial_phot):

    # # generate model
    prediction = model.mean_model(theta_best, obs=obs, sps=sps)
    pspec, pphot, pfrac = prediction

    xmin, xmax = np.min(wphot)*0.8, np.max(wphot)/0.8
    temp = np.interp(np.linspace(xmin,xmax,10000), wspec, initial_spec)
    ymin, ymax = temp.min()*0.8, temp.max()/0.4
    figure(figsize=(16,8))

    # plot Data, best fit model, and old models
    loglog(wspec, initial_spec, label='Old model spectrum',
        lw=0.7, color='gray', alpha=0.5)
    errorbar(wphot, initial_phot, label='Old model Photometry', 
            marker='s', markersize=10, alpha=0.6, ls='', lw=3, 
            markerfacecolor='none', markeredgecolor='gray', 
            markeredgewidth=3)
    loglog(wspec, pspec, label='Model spectrum', 
        lw=0.7, color='slateblue', alpha=0.7)
    errorbar(wphot, pphot, label='Model photometry', 
            marker='s', markersize=10, alpha=0.8, ls='', lw=3,
            markerfacecolor='none', markeredgecolor='slateblue', 
            markeredgewidth=3)
    errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'],
            label='Observed photometry', 
            marker='o', markersize=10, alpha=0.8, ls='', lw=3, 
            ecolor='tomato', markerfacecolor='none', markeredgecolor='tomato', 
            markeredgewidth=3)

    # plot filter transmission curves
    for f in obs['filters']:
        w, t = f.wavelength.copy(), f.transmission.copy()
        t = t / t.max()
        t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
        loglog(w, t, lw=3, color='gray', alpha=0.7)

    # Prettify
    xlabel('Wavelength [A]')
    ylabel('Flux Density [maggies]')
    xlim([xmin, xmax])
    ylim([10e-11, ymax])
    legend(loc='best', fontsize=20)
    tight_layout()

In [182]:
def plot_MCMC(run_params, obs, wphot, wspec, initial_spec, model, sps, lnprobfn, theta_best, galaxy_name):
    from prospect.fitting import fit_model

    #establish bounds
    xmin, xmax = np.min(wphot)*0.8, np.max(wphot)/0.8
    temp = np.interp(np.linspace(xmin,xmax,10000), wspec, initial_spec)
    ymin, ymax = temp.min()*0.8, temp.max()/0.4

    output = fit_model(obs, model, sps, lnprobfn=lnprobfn, **run_params)
    print('done emcee in {0}s'.format(output["sampling"][1]))

    from prospect.io import write_results as writer
    hfile = "demo_emcee_mcmc.h5"
    if os.path.isfile(hfile): os.remove(hfile)
    writer.write_hdf5(hfile, run_params, model, obs,
                    output["sampling"][0], output["optimization"][0],
                    tsample=output["sampling"][1],
                    toptimize=output["optimization"][1])

    import prospect.io.read_results as reader
    results_type = "emcee" # | "dynesty"
    # grab results (dictionary), the obs dictionary, and our corresponding models
    # When using parameter files set `dangerous=True`
    result, obs, _ = reader.results_from(hfile.format(results_type), dangerous=False)

    if results_type == "emcee":
        chosen = np.random.choice(result["run_params"]["nwalkers"], size=10, replace=False)
        tracefig = reader.traceplot(result, figsize=(20,10), chains=chosen)
    else:
        tracefig = reader.traceplot(result, figsize=(20,10))

    # maximum a posteriori (of the locations visited by the MCMC sampler)
    imax = np.argmax(result['lnprobability'])
    if results_type == "emcee":
        i, j = np.unravel_index(imax, result['lnprobability'].shape)
        theta_max = result['chain'][i, j, :].copy()
        thin = 5
    else:
        theta_max = result["chain"][imax, :]
        thin = 1

    print('Optimization value: {}'.format(theta_best))
    print('MAP value: {}'.format(theta_max))
    cornerfig = reader.subcorner(result, start=0, thin=thin, truths=theta_best, 
                                fig=subplots(5,5,figsize=(27,27))[0])

    # randomly chosen parameters from chain
    randint = np.random.randint
    if results_type == "emcee":
        nwalkers, niter = run_params['nwalkers'], run_params['niter']
        theta = result['chain'][randint(nwalkers), randint(niter)]
    else:
        theta = result["chain"][randint(len(result["chain"]))]

    # generate models
    # sps = reader.get_sps(result)  # this works if using parameter files
    mspec, mphot, mextra = model.mean_model(theta, obs, sps=sps)
    mspec_map, mphot_map, _ = model.mean_model(theta_max, obs, sps=sps)

    # Make plot of data and model
    figure(figsize=(16,8))

    loglog(wspec, mspec, label='Model spectrum (random draw)',
        lw=0.7, color='navy', alpha=0.7)
    loglog(wspec, mspec_map, label='Model spectrum (MAP)',
        lw=0.7, color='green', alpha=0.7)
    errorbar(wphot, mphot, label='Model photometry (random draw)',
            marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
            markerfacecolor='none', markeredgecolor='blue', 
            markeredgewidth=3)
    errorbar(wphot, mphot_map, label='Model photometry (MAP)',
            marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
            markerfacecolor='none', markeredgecolor='green', 
            markeredgewidth=3)
    errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'], 
            label='Observed photometry', ecolor='red', 
            marker='o', markersize=10, ls='', lw=3, alpha=0.8, 
            markerfacecolor='none', markeredgecolor='red', 
            markeredgewidth=3)

    # plot transmission curves
    for f in obs['filters']:
        w, t = f.wavelength.copy(), f.transmission.copy()
        t = t / t.max()
        t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
        loglog(w, t, lw=3, color='gray', alpha=0.7)

    xlabel('Wavelength [A]')
    ylabel('Flux Density [maggies]')
    xlim([xmin, xmax])
    ylim([10e-11, ymax])
    legend(loc='best', fontsize=20)
    tight_layout()

    file = open(FRB_Name+' Results.txt', 'a')
    file.write('\n\nSED fit results for '+galaxy_name+': \n\n')
    file.write('       Mass          Log(ZSol)  Dust2  TAge Log(Tau)')
    file.write('\n-1sig   ')
    data = np.percentile(result['chain'], axis=(0,1), q=16)
    for value in data:
        file.write(str("{:.4f}".format(value))+' ')
    file.write('\nMedian   ')
    data = np.percentile(result['chain'], axis=(0,1), q=50)
    for value in data:
        file.write(str("{:.4f}".format(value))+' ')
    file.write('\n+1sig   ')
    data = np.percentile(result['chain'], axis=(0,1), q=64)
    for value in data:
        file.write(str("{:.4f}".format(value))+' ')
    file.close()

    print(np.percentile(result['chain'], axis=(0,1), q=16))
    print(np.percentile(result['chain'], axis=(0,1), q=50))
    print(np.percentile(result['chain'], axis=(0,1), q=84))


In [183]:
for index in range(0, len(detections)):

    galaxy_name = detections.at[index, 'Name']
    galaxy_separation = detections.at[index, 'Separation']
    galaxy_magnitudes = []
    galaxy_errors = []
    for filter in ['g', 'r', 'i', 'z', 'y']:
        galaxy_magnitudes.append(detections.at[index, filter])
        galaxy_errors.append(detections.at[index, 'err_'+filter])
    galaxy_redshift = detections.at[index, 'zphot']

    run_params = {}
    run_params["snr"] = 10.0
    # run_params["ldist"] = 10.0

    # Build the obs dictionary using the meta-parameters
    obs = build_obs(**run_params)

    # --- Plot the Data ----
    # This is why we stored these...
    wphot = obs["phot_wave"]

    # plot_SED(wphot, obs)

    from prospect.models import priors
    mass_param = {"name": "mass",
                # The mass parameter here is a scalar, so it has N=1
                "N": 1,
                # We will be fitting for the mass, so it is a free parameter
                "isfree": True,
                # This is the initial value. For fixed parameters this is the
                # value that will always be used. 
                "init": 1e8,
                # This sets the prior probability for the parameter
                "prior": priors.LogUniform(mini=1e6, maxi=1e12),
                # this sets the initial dispersion to use when generating 
                # clouds of emcee "walkers".  It is not required, but can be very helpful.
                "init_disp": 1e6, 
                # this sets the minimum dispersion to use when generating 
                #clouds of emcee "walkers".  It is not required, but can be useful if 
                # burn-in rounds leave the walker distribution too narrow for some reason.
                "disp_floor": 1e6, 
                # This is not required, but can be helpful
                "units": "solar masses formed",
                }

    from prospect.models.templates import TemplateLibrary

    run_params["object_redshift"] = galaxy_redshift
    run_params["fixed_metallicity"] = None
    run_params["add_duste"] = True

    model = build_model(**run_params)

    run_params["zcontinuous"] = 1

    sps = build_sps(**run_params)

    # Generate the model SED at the initial value of theta
    theta = model.theta.copy()
    initial_spec, initial_phot, initial_mfrac = model.sed(theta, obs=obs, sps=sps)
    title_text = ','.join(["{}={}".format(p, model.params[p][0]) 
                        for p in model.free_params])

    a = 1.0 + model.params.get('zred', 0.0) # cosmological redshifting
    # photometric effective wavelengths
    wphot = obs["phot_wave"]
    # spectroscopic wavelengths
    if obs["wavelength"] is None:
        # *restframe* spectral wavelengths, since obs["wavelength"] is None
        wspec = sps.wavelengths
        wspec *= a #redshift them
    else:
        wspec = obs["wavelength"]

    # plot_prelim_fit(wphot, wspec, initial_spec, obs)

    from prospect.likelihood import lnlike_spec, lnlike_phot, write_log

    verbose = False
    run_params["verbose"] = verbose

    from prospect.fitting import lnprobfn

    # Here we will run all our building functions
    obs = build_obs(**run_params)
    sps = build_sps(**run_params)
    model = build_model(**run_params)

    # For fsps based sources it is useful to 
    # know which stellar isochrone and spectral library
    # we are using
    # print(sps.ssp.libraries)

    from prospect.fitting import fit_model

    # --- start minimization ----
    run_params["dynesty"] = False
    run_params["emcee"] = False
    run_params["optimize"] = True
    run_params["min_method"] = 'lm'
    # We'll start minimization from "nmin" separate places, 
    # the first based on the current values of each parameter and the 
    # rest drawn from the prior.  Starting from these extra draws 
    # can guard against local minima, or problems caused by 
    # starting at the edge of a prior (e.g. dust2=0.0)
    run_params["nmin"] = 2

    output = fit_model(obs, model, sps, lnprobfn=lnprobfn, **run_params)

    # print("Done optmization in {}s".format(output["optimization"][1]))

    # print(model.theta)

    (results, topt) = output["optimization"]
    # # Find which of the minimizations gave the best result, 
    # # and use the parameter vector for that minimization
    ind_best = np.argmin([r.cost for r in results])
    # print(ind_best)
    theta_best = results[ind_best].x.copy()

    # theta_mass = theta_best[0]
    # max_mass = theta_mass*10
    # virial_radius = (max_mass**3)
    
    # count = 0
    # if virial_radius < galaxy_separation:
    #     count+=1

    # print(count)
    

    #plot_least_squares(model, theta_best, obs, sps, wspec, wphot, initial_spec, initial_phot)
    
    # run_params["optimize"] = False
    # run_params["emcee"] = True
    # run_params["dynesty"] = False
    # # Number of emcee walkers
    # run_params["nwalkers"] = 128
    # # Number of iterations of the MCMC sampling
    # run_params["niter"] = 512
    # # Number of iterations in each round of burn-in
    # # After each round, the walkers are reinitialized based on the 
    # # locations of the highest probablity half of the walkers.
    # run_params["nburn"] = [16, 32, 64]

    # plot_MCMC(run_params, obs, wphot, wspec, initial_spec, model, sps, lnprobfn, theta_best, galaxy_name)

    
    

[3890. 3900. 3910. 3920. 3930. 3940. 3950. 3960. 3970. 3980. 3990. 4000.
 4010. 4020. 4030. 4040. 4050. 4060. 4070. 4080. 4090. 4100. 4110. 4120.
 4130. 4140. 4150. 4160. 4170. 4180. 4190. 4200. 4210. 4220. 4230. 4240.
 4250. 4260. 4270. 4280. 4290. 4300. 4310. 4320. 4330. 4340. 4350. 4360.
 4370. 4380. 4390. 4400. 4410. 4420. 4430. 4440. 4450. 4460. 4470. 4480.
 4490. 4500. 4510. 4520. 4530. 4540. 4550. 4560. 4570. 4580. 4590. 4600.
 4610. 4620. 4630. 4640. 4650. 4660. 4670. 4680. 4690. 4700. 4710. 4720.
 4730. 4740. 4750. 4760. 4770. 4780. 4790. 4800. 4810. 4820. 4830. 4840.
 4850. 4860. 4870. 4880. 4890. 4900. 4910. 4920. 4930. 4940. 4950. 4960.
 4970. 4980. 4990. 5000. 5010. 5020. 5030. 5040. 5050. 5060. 5070. 5080.
 5090. 5100. 5110. 5120. 5130. 5140. 5150. 5160. 5170. 5180. 5190. 5200.
 5210. 5220. 5230. 5240. 5250. 5260. 5270. 5280. 5290. 5300. 5310. 5320.
 5330. 5340. 5350. 5360. 5370. 5380. 5390. 5400. 5410. 5420. 5430. 5440.
 5450. 5460. 5470. 5480. 5490. 5500. 5510. 5520. 55

/home/rns/.local/lib/python3.8/site-packages/prospect/models/priors.py:117: RuntimeWarning: divide by zero encountered in log
  lnp = np.log(p)


0
[3890. 3900. 3910. 3920. 3930. 3940. 3950. 3960. 3970. 3980. 3990. 4000.
 4010. 4020. 4030. 4040. 4050. 4060. 4070. 4080. 4090. 4100. 4110. 4120.
 4130. 4140. 4150. 4160. 4170. 4180. 4190. 4200. 4210. 4220. 4230. 4240.
 4250. 4260. 4270. 4280. 4290. 4300. 4310. 4320. 4330. 4340. 4350. 4360.
 4370. 4380. 4390. 4400. 4410. 4420. 4430. 4440. 4450. 4460. 4470. 4480.
 4490. 4500. 4510. 4520. 4530. 4540. 4550. 4560. 4570. 4580. 4590. 4600.
 4610. 4620. 4630. 4640. 4650. 4660. 4670. 4680. 4690. 4700. 4710. 4720.
 4730. 4740. 4750. 4760. 4770. 4780. 4790. 4800. 4810. 4820. 4830. 4840.
 4850. 4860. 4870. 4880. 4890. 4900. 4910. 4920. 4930. 4940. 4950. 4960.
 4970. 4980. 4990. 5000. 5010. 5020. 5030. 5040. 5050. 5060. 5070. 5080.
 5090. 5100. 5110. 5120. 5130. 5140. 5150. 5160. 5170. 5180. 5190. 5200.
 5210. 5220. 5230. 5240. 5250. 5260. 5270. 5280. 5290. 5300. 5310. 5320.
 5330. 5340. 5350. 5360. 5370. 5380. 5390. 5400. 5410. 5420. 5430. 5440.
 5450. 5460. 5470. 5480. 5490. 5500. 5510. 5520. 

In [180]:
print(theta_best[0])
print((np.percentile(result['chain'], axis=(0,1), q=50)))

20044311.569958072
[ 9.41154231e+07 -3.48870832e-01  1.28619700e+00  9.99749820e+00
  3.20415709e-01]


Let's run all of these functions to fit the SEDs:

In [ ]:
# for i in range(0, len(detections)):
for galaxy_index in range(0, 1):

    galaxy_magnitudes = []
    galaxy_errors = []
    for filter in ['g', 'r', 'i', 'z', 'y']:
        galaxy_magnitudes.append(detections.at[galaxy_index, filter])
        galaxy_errors.append(detections.at[galaxy_index, 'err_'+filter])
    galaxy_redshift = detections.at[galaxy_index, 'zphot']

    run_params = {}
    run_params["snr"] = 10.0
    # run_params["ldist"] = 10.0

    # Build the obs dictionary using the meta-parameters
    obs = build_obs(**run_params)

    # --- Plot the Data ----
    # This is why we stored these...
    wphot = obs["phot_wave"]

    # # establish bounds
    xmin, xmax = np.min(wphot)*0.8, np.max(wphot)/0.8
    ymin, ymax = obs["maggies"].min()*0.8, obs["maggies"].max()/0.4
    figure(figsize=(16,8))

    # plot all the data
    plot(wphot, obs['maggies'],
        label='All observed photometry',
        marker='o', markersize=12, alpha=0.8, ls='', lw=3,
        color='slateblue')

    # overplot only the data we intend to fit
    mask = obs["phot_mask"]
    errorbar(wphot[mask], obs['maggies'][mask], 
            yerr=obs['maggies_unc'][mask], 
            label='Photometry to fit',
            marker='o', markersize=8, alpha=0.8, ls='', lw=3,
            ecolor='tomato', markerfacecolor='none', markeredgecolor='tomato', 
            markeredgewidth=3)

    # plot Filters
    for f in obs['filters']:
        w, t = f.wavelength.copy(), f.transmission.copy()
        t = t / t.max()
        t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
        loglog(w, t, lw=3, color='gray', alpha=0.7)

    # prettify
    xlabel('Wavelength [A]')
    ylabel('Flux Density [maggies]')
    xlim([xmin, xmax])
    ylim([10e-11, ymax])
    xscale("log")
    yscale("log")
    legend(loc='best', fontsize=20)
    tight_layout()

    from prospect.models import priors
    mass_param = {"name": "mass",
                # The mass parameter here is a scalar, so it has N=1
                "N": 1,
                # We will be fitting for the mass, so it is a free parameter
                "isfree": True,
                # This is the initial value. For fixed parameters this is the
                # value that will always be used. 
                "init": 1e8,
                # This sets the prior probability for the parameter
                "prior": priors.LogUniform(mini=1e6, maxi=1e12),
                # this sets the initial dispersion to use when generating 
                # clouds of emcee "walkers".  It is not required, but can be very helpful.
                "init_disp": 1e6, 
                # this sets the minimum dispersion to use when generating 
                #clouds of emcee "walkers".  It is not required, but can be useful if 
                # burn-in rounds leave the walker distribution too narrow for some reason.
                "disp_floor": 1e6, 
                # This is not required, but can be helpful
                "units": "solar masses formed",
                }

    from prospect.models.templates import TemplateLibrary

    run_params["object_redshift"] = galaxy_redshift
    run_params["fixed_metallicity"] = None
    run_params["add_duste"] = True

    model = build_model(**run_params)
    print(model)

    run_params["zcontinuous"] = 1

    sps = build_sps(**run_params)

    # Generate the model SED at the initial value of theta
    theta = model.theta.copy()
    initial_spec, initial_phot, initial_mfrac = model.sed(theta, obs=obs, sps=sps)
    title_text = ','.join(["{}={}".format(p, model.params[p][0]) 
                        for p in model.free_params])

    a = 1.0 + model.params.get('zred', 0.0) # cosmological redshifting
    # photometric effective wavelengths
    wphot = obs["phot_wave"]
    # spectroscopic wavelengths
    if obs["wavelength"] is None:
        # *restframe* spectral wavelengths, since obs["wavelength"] is None
        wspec = sps.wavelengths
        wspec *= a #redshift them
    else:
        wspec = obs["wavelength"]

    #establish bounds
    xmin, xmax = np.min(wphot)*0.8, np.max(wphot)/0.8
    temp = np.interp(np.linspace(xmin,xmax,10000), wspec, initial_spec)
    ymin, ymax = temp.min()*0.8, temp.max()/0.4
    figure(figsize=(16,8))

    # plot model + data
    loglog(wspec, initial_spec, label='Model spectrum', 
        lw=0.7, color='navy', alpha=0.7)
    errorbar(wphot, initial_phot, label='Model photometry', 
            marker='s',markersize=10, alpha=0.8, ls='', lw=3,
            markerfacecolor='none', markeredgecolor='blue', 
            markeredgewidth=3)
    errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'], 
            label='Observed photometry',
            marker='o', markersize=10, alpha=0.8, ls='', lw=3,
            ecolor='red', markerfacecolor='none', markeredgecolor='red', 
            markeredgewidth=3)
    title(title_text)

    # plot Filters
    for f in obs['filters']:
        w, t = f.wavelength.copy(), f.transmission.copy()
        t = t / t.max()
        t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
        loglog(w, t, lw=3, color='gray', alpha=0.7)

    #prettify
    xlabel('Wavelength [A]')
    ylabel('Flux Density [maggies]')
    xlim([xmin, xmax])
    ylim([10e-11, ymax])
    legend(loc='best', fontsize=20)
    tight_layout()

    from prospect.likelihood import lnlike_spec, lnlike_phot, write_log

    verbose = False
    run_params["verbose"] = verbose

    from prospect.fitting import lnprobfn

    # Here we will run all our building functions
    obs = build_obs(**run_params)
    sps = build_sps(**run_params)
    model = build_model(**run_params)

    # For fsps based sources it is useful to 
    # know which stellar isochrone and spectral library
    # we are using
    print(sps.ssp.libraries)

    from prospect.fitting import fit_model

    # --- start minimization ----
    run_params["dynesty"] = False
    run_params["emcee"] = False
    run_params["optimize"] = True
    run_params["min_method"] = 'lm'
    # We'll start minimization from "nmin" separate places, 
    # the first based on the current values of each parameter and the 
    # rest drawn from the prior.  Starting from these extra draws 
    # can guard against local minima, or problems caused by 
    # starting at the edge of a prior (e.g. dust2=0.0)
    run_params["nmin"] = 2

    output = fit_model(obs, model, sps, lnprobfn=lnprobfn, **run_params)

    # print("Done optmization in {}s".format(output["optimization"][1]))

    # print(model.theta)

    (results, topt) = output["optimization"]
    # # Find which of the minimizations gave the best result, 
    # # and use the parameter vector for that minimization
    ind_best = np.argmin([r.cost for r in results])
    # print(ind_best)
    theta_best = results[ind_best].x.copy()
    print(theta_best)

    # # generate model
    prediction = model.mean_model(theta_best, obs=obs, sps=sps)
    pspec, pphot, pfrac = prediction

    figure(figsize=(16,8))

    # plot Data, best fit model, and old models
    loglog(wspec, initial_spec, label='Old model spectrum',
        lw=0.7, color='gray', alpha=0.5)
    errorbar(wphot, initial_phot, label='Old model Photometry', 
            marker='s', markersize=10, alpha=0.6, ls='', lw=3, 
            markerfacecolor='none', markeredgecolor='gray', 
            markeredgewidth=3)
    loglog(wspec, pspec, label='Model spectrum', 
        lw=0.7, color='slateblue', alpha=0.7)
    errorbar(wphot, pphot, label='Model photometry', 
            marker='s', markersize=10, alpha=0.8, ls='', lw=3,
            markerfacecolor='none', markeredgecolor='slateblue', 
            markeredgewidth=3)
    errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'],
            label='Observed photometry', 
            marker='o', markersize=10, alpha=0.8, ls='', lw=3, 
            ecolor='tomato', markerfacecolor='none', markeredgecolor='tomato', 
            markeredgewidth=3)

    # plot filter transmission curves
    for f in obs['filters']:
        w, t = f.wavelength.copy(), f.transmission.copy()
        t = t / t.max()
        t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
        loglog(w, t, lw=3, color='gray', alpha=0.7)

    # Prettify
    xlabel('Wavelength [A]')
    ylabel('Flux Density [maggies]')
    xlim([xmin, xmax])
    ylim([10e-11, ymax])
    legend(loc='best', fontsize=20)
    tight_layout()

    # Set this to False if you don't want to do another optimization
    # before emcee sampling (but note that the "optimization" entry 
    # in the output dictionary will be (None, 0.) in this case)
    # If set to true then another round of optmization will be performed 
    # before sampling begins and the "optmization" entry of the output
    # will be populated.
    run_params["optimize"] = False
    run_params["emcee"] = True
    run_params["dynesty"] = False
    # Number of emcee walkers
    run_params["nwalkers"] = 128
    # Number of iterations of the MCMC sampling
    run_params["niter"] = 512
    # Number of iterations in each round of burn-in
    # After each round, the walkers are reinitialized based on the 
    # locations of the highest probablity half of the walkers.
    run_params["nburn"] = [16, 32, 64]

    output = fit_model(obs, model, sps, lnprobfn=lnprobfn, **run_params)
    print('done emcee in {0}s'.format(output["sampling"][1]))

    from prospect.io import write_results as writer
    hfile = "demo_emcee_mcmc.h5"
    if os.path.isfile(hfile): os.remove(hfile)
    writer.write_hdf5(hfile, run_params, model, obs,
                    output["sampling"][0], output["optimization"][0],
                    tsample=output["sampling"][1],
                    toptimize=output["optimization"][1])

    import prospect.io.read_results as reader
    results_type = "emcee" # | "dynesty"
    # grab results (dictionary), the obs dictionary, and our corresponding models
    # When using parameter files set `dangerous=True`
    result, obs, _ = reader.results_from(hfile.format(results_type), dangerous=False)

    # from prospect import prospect_args
    # # - Parser with default arguments -
    # parser = prospect_args.get_parser()
    # # - Add custom arguments -
    # parser.add_argument('--add_duste', action="store_true",
    #                     help="If set, add dust emission to the model.")
    # parser.add_argument('--ldist', type=float, default=10,
    #                     help=("Luminosity distance in Mpc. Defaults to 10"
    #                         "(for case of absolute mags)"))
    # args, _ = parser.parse_known_args()
    # cli_run_params = vars(args)
    # print(cli_run_params)

    # import prospect.io.read_results as reader
    # results_type = "emcee" # | "dynesty"
    # # grab results (dictionary), the obs dictionary, and our corresponding models
    # # When using parameter files set `dangerous=True`
    # result, obs, _ = reader.results_from("demo_{}_mcmc.h5".format(results_type), dangerous=False)

    #The following commented lines reconstruct the model and sps object, 
    # if a parameter file continaing the `build_*` methods was saved along with the results
    #model = reader.get_model(result)
    #sps = reader.get_sps(result)

    # let's look at what's stored in the `result` dictionary
    # print(result.keys())

    if results_type == "emcee":
        chosen = np.random.choice(result["run_params"]["nwalkers"], size=10, replace=False)
        tracefig = reader.traceplot(result, figsize=(20,10), chains=chosen)
    else:
        tracefig = reader.traceplot(result, figsize=(20,10))

    # maximum a posteriori (of the locations visited by the MCMC sampler)
    imax = np.argmax(result['lnprobability'])
    if results_type == "emcee":
        i, j = np.unravel_index(imax, result['lnprobability'].shape)
        theta_max = result['chain'][i, j, :].copy()
        thin = 5
    else:
        theta_max = result["chain"][imax, :]
        thin = 1

    print('Optimization value: {}'.format(theta_best))
    print('MAP value: {}'.format(theta_max))
    cornerfig = reader.subcorner(result, start=0, thin=thin, truths=theta_best, 
                                fig=subplots(5,5,figsize=(27,27))[0])

    # randomly chosen parameters from chain
    randint = np.random.randint
    if results_type == "emcee":
        nwalkers, niter = run_params['nwalkers'], run_params['niter']
        theta = result['chain'][randint(nwalkers), randint(niter)]
    else:
        theta = result["chain"][randint(len(result["chain"]))]

    # generate models
    # sps = reader.get_sps(result)  # this works if using parameter files
    mspec, mphot, mextra = model.mean_model(theta, obs, sps=sps)
    mspec_map, mphot_map, _ = model.mean_model(theta_max, obs, sps=sps)

    # Make plot of data and model
    figure(figsize=(16,8))

    loglog(wspec, mspec, label='Model spectrum (random draw)',
        lw=0.7, color='navy', alpha=0.7)
    loglog(wspec, mspec_map, label='Model spectrum (MAP)',
        lw=0.7, color='green', alpha=0.7)
    errorbar(wphot, mphot, label='Model photometry (random draw)',
            marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
            markerfacecolor='none', markeredgecolor='blue', 
            markeredgewidth=3)
    errorbar(wphot, mphot_map, label='Model photometry (MAP)',
            marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
            markerfacecolor='none', markeredgecolor='green', 
            markeredgewidth=3)
    errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'], 
            label='Observed photometry', ecolor='red', 
            marker='o', markersize=10, ls='', lw=3, alpha=0.8, 
            markerfacecolor='none', markeredgecolor='red', 
            markeredgewidth=3)

    # plot transmission curves
    for f in obs['filters']:
        w, t = f.wavelength.copy(), f.transmission.copy()
        t = t / t.max()
        t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
        loglog(w, t, lw=3, color='gray', alpha=0.7)

    xlabel('Wavelength [A]')
    ylabel('Flux Density [maggies]')
    xlim([xmin, xmax])
    ylim([10e-11, ymax])
    legend(loc='best', fontsize=20)
    tight_layout()

    file = open(FRB_Name+' Results.txt', 'a')
    file.write('\n\nSED fit results for '+detections.at[galaxy_index, 'Name']+': \n\n')
    file.write('       Mass          Log(ZSol)  Dust2  TAge Log(Tau)')
    file.write('\n-1sig   ')
    data = np.percentile(result['chain'], axis=(0,1), q=16)
    for value in data:
        file.write(str("{:.4f}".format(value))+' ')
    file.write('\nMedian   ')
    data = np.percentile(result['chain'], axis=(0,1), q=50)
    for value in data:
        file.write(str("{:.4f}".format(value))+' ')
    file.write('\n+1sig   ')
    data = np.percentile(result['chain'], axis=(0,1), q=64)
    for value in data:
        file.write(str("{:.4f}".format(value))+' ')
    file.close()


In [ ]:
figure(figsize=(16,8))

    # loglog(wspec, mspec, label='Model spectrum (random draw)',
    #     lw=0.7, color='navy', alpha=0.7)
loglog(wspec, mspec_map, label='Model spectrum (MAP)',
    lw=0.7, color='green', alpha=0.7)
# errorbar(wphot, mphot, label='Model photometry (random draw)',
#         marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
#         markerfacecolor='none', markeredgecolor='blue', 
#         markeredgewidth=3)
errorbar(wphot, mphot_map, label='Model photometry (MAP)',
        marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
        markerfacecolor='none', markeredgecolor='green', 
        markeredgewidth=3)
errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'], 
        label='Observed photometry', ecolor='red', 
        marker='o', markersize=10, ls='', lw=3, alpha=0.8, 
        markerfacecolor='none', markeredgecolor='red', 
        markeredgewidth=3)

# plot transmission curves
for f in obs['filters']:
    w, t = f.wavelength.copy(), f.transmission.copy()
    t = t / t.max()
    t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
    loglog(w, t, lw=3, color='gray', alpha=0.7)

xlabel('Wavelength [A]')
ylabel('Flux Density [maggies]')
xlim([xmin, xmax])
ylim([10e-11, ymax])
legend(loc='best', fontsize=20)
tight_layout()

In [ ]:
figure(figsize=(16,8))

# plot Data, best fit model, and old models
# loglog(wspec, initial_spec, label='Old model spectrum',
#     lw=0.7, color='gray', alpha=0.5)
# errorbar(wphot, initial_phot, label='Old model Photometry', 
#         marker='s', markersize=10, alpha=0.6, ls='', lw=3, 
#         markerfacecolor='none', markeredgecolor='gray', 
#         markeredgewidth=3)
loglog(wspec, pspec, label='Model spectrum', 
    lw=0.7, color='slateblue', alpha=1)
errorbar(wphot, pphot, label='Model photometry', 
        marker='s', markersize=10, alpha=1, ls='', lw=3,
        markerfacecolor='none', markeredgecolor='slateblue', 
        markeredgewidth=3)
errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'],
        label='Observed photometry', 
        marker='o', markersize=10, alpha=1, ls='', lw=3, 
        ecolor='tomato', markerfacecolor='none', markeredgecolor='tomato', 
        markeredgewidth=3)

# plot filter transmission curves
for f in obs['filters']:
    w, t = f.wavelength.copy(), f.transmission.copy()
    t = t / t.max()
    t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
    loglog(w, t, lw=3, color='gray', alpha=1)

# Prettify
xlabel('Wavelength [A]')
ylabel('Flux Density [maggies]')
xlim([xmin, xmax])
ylim([10e-11, ymax])
legend(loc='best', fontsize=20)
tight_layout()